In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 0 = all logs, 1 = filter INFO, 2 = filter WARNING, 3 = filter ERROR
import numpy as np # linear algebra
import pandas as pd 
import nltk
nltk.download('punkt')
from nltk import word_tokenize
nltk.download('stopwords')
import re
import string 
import emoji
import random
import arabic_reshaper
from bidi.algorithm import get_display
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from transformers import AutoTokenizer

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

# This part is commented because currently I'm training locally

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Retr0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Retr0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
sample_path = './arabic_dataset/'
df_train =pd.read_csv(sample_path + 'train.csv')

In [6]:
df_train.head()

,correct,incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...


In [7]:
df_train.shape

(18350, 2)

In [9]:
df_valid =pd.read_csv(sample_path + "valid.csv")

In [10]:
df_valid.head()

,correct,incorrect
0,لا يعتقد البعض بان ضغوطات الغرب علي سوريا هي ش...,لا يعتقد البعض بان ضغوطات الغرب علي سوريا هي ش...
1,انا متاكد ان التعليمات تاتي من الغرب لزعماء ال...,انا متاكد ان التعليمات تاتي من الغرب لزعماء ال...
2,اناشد الاخوه في مصر ارض الكنانه ان يتقوا الله ...,اناشد الاخوه في مصر ارض الكنانه ان يتقوا الله ...
3,علي من ميلانو يقول : ان سوريا بغالبيتها التي ت...,علي من ميلانوا يقول ان سوريا بغالبيته التي تبل...
4,قسما بالله ، انا اعرف اهل حمص ، اليوم شيعنا خي...,قسما بالله انا اعرف اهل حمص اليوم شيعنا خيره ا...


In [11]:
df_valid.shape

(2293, 2)

In [12]:
print(df_valid.isnull().sum())
print(df_train.isnull().sum())

correct      0
incorrect    0
dtype: int64
correct      0
incorrect    0
dtype: int64


In [13]:
print(df_valid.duplicated().sum())
print(df_train.duplicated().sum())

10
23


In [14]:
df_valid = df_valid.drop_duplicates()
df_train = df_train.drop_duplicates()

In [15]:
print(df_valid.duplicated().sum())
print(df_train.duplicated().sum())

0
0


In [16]:
import numpy as np
import random

# Parameters
num_nans = 50  # total number of NaNs to insert

# Get DataFrame shape
rows, cols = df_valid.shape
columns = df_valid.columns.tolist()

# Add NaNs at random (row, column) positions
for _ in range(num_nans):
    rand_row = random.randint(0, rows - 1)
    rand_col = random.choice(columns)
    df_valid.at[rand_row, rand_col] = np.nan
    df_train.at[rand_row, rand_col] = np.nan


In [17]:
print(df_valid.isnull().sum())
print(df_train.isnull().sum())

correct      23
incorrect    27
dtype: int64
correct      23
incorrect    27
dtype: int64


In [18]:
df_valid = df_valid.dropna()
df_train = df_train.dropna()

In [19]:
print(df_valid.isnull().sum())
print(df_train.isnull().sum())

correct      0
incorrect    0
dtype: int64
correct      0
incorrect    0
dtype: int64


In [20]:
df = pd.concat([df_train, df_valid])
df

,correct,incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...
...,...,...
2288,اعتقد ان اي مسلم من المستحيل ان يفعل هذا ، سوا...,اعتقد ان اي مسلم من المستحيل ان يفعل هذا سواء ...
2289,( . . وهو يصلح جدران بيته ويصبغها ، وحينما يست...,( . . وهو يصلح جدران بيته ويصبغها وحينما يستعص...
2290,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...,انا استغرب ممن ينكر علي الجزيره ويصفها باثاره ...
2291,تصرف طبيعي ، اي انسان في امريكا يتعرض بالنقد ل...,تصرف طبيعي اي انسان في امريكا يتعرض بالنقد للي...


In [21]:
stop = set(nltk.corpus.stopwords.words("arabic"))

# Text length analysis 
def avg_word(sentence):
    words = sentence.split()
    if len(words) == 0:
        return 0
    return (sum(len(word) for word in words)/len(words))

# Counting emoji
def emoji_counter(sentence):
    return emoji.emoji_count(sentence)

df['word_count_correct'] = df['correct'].apply(lambda x: len(str(x).split(" ")))
df['word_count_incorrect'] = df['incorrect'].apply(lambda x: len(str(x).split(" ")))
df['stopwords_correct'] = df['correct'].apply(lambda x: len([x for x in x.split() if x in stop]))
df['stopwords_incorrect'] = df['incorrect'].apply(lambda x: len([x for x in x.split() if x in stop]))
df['emoji_count_correct'] = df['correct'].apply(lambda x: emoji_counter(x))
df['emoji_count_incorrect'] = df['incorrect'].apply(lambda x: emoji_counter(x))
df.head()

,correct,incorrect,word_count_correct,word_count_incorrect,stopwords_correct,stopwords_incorrect,emoji_count_correct,emoji_count_incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...,65,59,12,8,0,0
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...,54,45,10,10,0,0
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,66,57,7,7,0,0
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...,70,62,10,10,0,0
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,66,64,11,11,0,0


In [22]:
df.describe() 

,word_count_correct,word_count_incorrect,stopwords_correct,stopwords_incorrect,emoji_count_correct,emoji_count_incorrect
count,20510.000000,20510.000000,20510.000000,20510.000000,20510.0,20510.0
mean,58.727353,54.250171,9.334812,9.741589,0.0,0.0
std,18.534074,17.581705,4.497467,5.002717,0.0,0.0
min,6.000000,6.000000,0.000000,0.000000,0.0,0.0
25%,51.000000,47.000000,7.000000,7.000000,0.0,0.0
50%,57.000000,52.000000,9.000000,9.000000,0.0,0.0
75%,63.000000,58.000000,11.000000,12.000000,0.0,0.0
max,484.000000,461.000000,132.000000,132.000000,0.0,0.0


In [23]:
# Function to add a random character to a word in Arabic text
def add_random_arabic_character(word):
    arabic_chars = "ابتثجحخدذرزسشصضطظعغفقكلمنهوي"
    random_char = random.choice(arabic_chars)
    return word + random_char

# Function to remove a random character from a word in Arabic text
def remove_random_arabic_character(word):
    if len(word) <= 1:
        return word  # If the word has only one character, return the same word
    random_index = random.randint(0, len(word) - 1)  # Choose a random index
    return word[:random_index] + word[random_index + 1:]  # Remove the character at the random index

In [24]:
def modify_text_all_words(text):
    words = text.split()
    modified_words = []
    for word in words:
        modify_choice = random.choice(["add", "remove"])
        if modify_choice == "add":
            modified_words.append(add_random_arabic_character(word))
        elif modify_choice == "remove":
            modified_words.append(remove_random_arabic_character(word))
    return ' '.join(modified_words)


In [30]:
df['new_incorrect'] = df['correct'].apply(modify_text_all_words)

In [31]:
df['new_incorrect']

0       سبحا اللهد ،ي الحكامك اعرب سيموتع عليك الكرسيو...
1       النر ت ل محالد انك شاءت الل . ن يءمنث باللهص و...
2       اليت ك شخصو يعتد ن بشارض ااسد سوفش يخرجن ن سور...
3       الاد وصابته -ت لحدح الن - غيرس مسوعبين اه ر ين...
4       لنظام الاسديش تجاوزظ حتيث م فله معمرس القذاي ف...
                              ...                        
2288    اعتقدخ انر ا مسلمد م المستحيلف انث يفل هذاز ، ...
2289    ( .ش . وه صلح جدان يته ويصبغهاث ، وحينماك يستع...
2290    اناغ استغربم ممنل ينر عل الجزيرهذ ويصفهاط باثا...
2291    تصرفو طيعي ، ي انسانس فيج امريكاه يتعرضخ بالقد...
2292    اسراءل سوفج لنخ تنهارع بهذهن السوله ،ز اا اا ت...
Name: new_incorrect, Length: 20510, dtype: object

In [32]:
df=df[["correct","new_incorrect"]]

In [33]:
arabicPunctuations = [".","#","$","//","?","=","'","_","-","'",";","\\","`","؛","<",">","(",")","*","&","^","%","]","[",",","ـ","،","/",":","؟",".","'","{","}","~","|","!","”","…","“","–"] # defining customized punctuation marks
englishPunctuations = string.punctuation # importing English punctuation marks
englishPunctuations = [word.strip() for word in englishPunctuations] # converting the English punctuation from a string to array for processing
punctuationsList = arabicPunctuations + englishPunctuations # creating a list of all punctuation marks

def removingPunctuation(text):
  cleanTweet = ''
  for i in text:
    if i not in arabicPunctuations:
      cleanTweet = cleanTweet + '' + i 
  return cleanTweet

df['correct'] = df['correct'].apply(lambda x: removingPunctuation(x))
df['new_incorrect'] = df['new_incorrect'].apply(lambda x: removingPunctuation(x))


df.sample(5)


,correct,new_incorrect
13783,جعجعه بس ما في طحن هذا هو مستوي الاخوان الحقي...,جععه بست ماض ف حن ذا هوش متوي الاخواني الحيقي...
11852,"مصيبه الاسلام في "" مسلمي "" هذا الزمان ان ليس ...","صيبه الاسلامغ فيا ""ز مسلميو "" ها الزمانك ن لي..."
17060,انا لله وانا اليه راجعون بن بله هو اب الفقرا...,اناز له ونا اليهي راجعونت ض بنخ بل هوذ ابص ال...
8790,انتم الذين تريدون الامم المتحده والدول ان تسكت...,انتمم الذي ريدون الاممر امتحده والدو ا تكت ع ج...
14627,لن نقبل بان تكون مصر مرتعا للاخوان ويحولوها ال...,ن نقبلض بانث تون صر مرتعام لاخوان ويحولوهاغ ال...


In [34]:
def clean(text):
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text)  
    # Remove numbers
    text = re.sub("\d+", " ", text)
    return text

df['correct'] = df['correct'].apply(lambda x:clean(x))
df['correct'] = df['correct'].apply(lambda x:clean(x))

df['new_incorrect'] = df['new_incorrect'].apply(lambda x:clean(x))
df['new_incorrect'] = df['new_incorrect'].apply(lambda x:clean(x))

In [35]:
# Initialize the Tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)
# Fit the tokenizer on the concatenated correct and incorrect texts
tokenizer.fit_on_texts(df['correct'])

In [36]:
# Convert the texts to sequences
correct_sequences = tokenizer.texts_to_sequences(df['correct'])
incorrect_sequences = tokenizer.texts_to_sequences(df['new_incorrect'])

In [37]:
# Calculate max sequence length for padding
max_sequence_length = max(
    max(len(seq) for seq in correct_sequences),
    max(len(seq) for seq in incorrect_sequences)
)

In [38]:
# Pad the sequences to uniform length
correct_sequences = pad_sequences(correct_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
incorrect_sequences = pad_sequences(incorrect_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [39]:
print(correct_sequences[:2]) 

[[  836     6   380    34 12801     4  1276 21656    44 18432  4941    72
     26    47  1220  2442    17   398 21657     1   213 12802  1319 18433
   6099   322  2350 21658     9   172    19 26301  6727   398  3027  1892
     12  2443 18434 12803     2  2781  1087  7508    97 12804  4942 26302
     31 12805  1276  4561  1709  2528 49000 18435     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0 

In [40]:
len(tokenizer.word_index)

100624

In [41]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    incorrect_sequences, correct_sequences, test_size=0.1, random_state=42)

In [42]:
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding

input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=128)(input_layer)
lstm_layer = LSTM(256, return_sequences=True)(embedding_layer)    #encider
output_layer = Dense(vocab_size, activation='softmax')(lstm_layer)     #decoder

model = Model(input_layer, output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 394)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 394, 128)       │    12,880,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 394, 256)       │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 394, 100625)    │    25,860,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 39,134,865 (149.29 MB)

 Trainable params: 39,134,865 (149.29 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=16)

Epoch 1/16


I0000 00:00:1747114916.670859     121 cuda_dnn.cc:529] Loaded cuDNN version 90300


1154/1154 ━━━━━━━━━━━━━━━━━━━━ 607s 523ms/step - accuracy: 0.8683 - loss: 2.0095 - val_accuracy: 0.8758 - val_loss: 1.1755
Epoch 2/16
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 603s 522ms/step - accuracy: 0.8762 - loss: 1.1502 - val_accuracy: 0.8761 - val_loss: 1.1686
Epoch 3/16
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 603s 523ms/step - accuracy: 0.8763 - loss: 1.1311 - val_accuracy: 0.8763 - val_loss: 1.1678
Epoch 4/16
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 603s 522ms/step - accuracy: 0.8765 - loss: 1.1162 - val_accuracy: 0.8764 - val_loss: 1.1755
Epoch 5/16
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 603s 522ms/step - accuracy: 0.8770 - loss: 1.1001 - val_accuracy: 0.8753 - val_loss: 1.1775
Epoch 6/16
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 602s 522ms/step - accuracy: 0.8775 - loss: 1.0806 - val_accuracy: 0.8761 - val_loss: 1.1790
Epoch 7/16
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 603s 522ms/step - accuracy: 0.8779 - loss: 1.0621 - val_accuracy: 0.8762 - val_loss: 1.1822
Epoch 8/16
1154/1154 ━━━━━━━━━━━━━━━━━━━━ 603s 522ms/step - accuracy: 0.8

In [ ]:
# Save your model
model.save('/kaggle/working/arabic_spelling_correction_model.h5')

In [ ]:
import os
os.listdir('/kaggle/working/')

['arabic_spelling_correction_model.h5', '.virtual_documents']

In [ ]:
import shutil

# Create a zip file of the saved model
shutil.make_archive('/kaggle/working/arabic_spelling_correction_model', 'zip', '/kaggle/working', 'arabic_spelling_correction_model.h5')

'/kaggle/working/arabic_spelling_correction_model.zip'

In [ ]:
model_path = "./arabic_spelling_correction_model.h5"
model = tf.keras.models.load_model(model_path)
model.summary()

In [ ]:
# Tokenize the input text
input_text = str(input())
input_sequence = tokenizer.texts_to_sequences([input_text])

# Pad the input sequence
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')

# Predict the corrected output
output_sequence = model.predict(padded_input_sequence)
corrected_output_sequence = np.argmax(output_sequence, axis=-1)

# Convert the corrected output sequence back to text
corrected_output_text = tokenizer.sequences_to_texts(corrected_output_sequence)

print("Corrected Output:", corrected_output_text)

In [57]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
# Assuming you already have a tokenizer object named 'tokenizer'

# Save the tokenizer
tokenizer_path = 'tokenizer.pkl'
with open(tokenizer_path, 'wb') as f:
    pickle.dump(tokenizer, f)

# Load the tokenizer
tokenizer_path = 'tokenizer.pkl'
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

In [58]:
def predict(input_text):
    input_sequence = tokenizer.texts_to_sequences([input_text])

    # Pad the input sequence
    padded_input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')

    # Predict the corrected output
    output_sequence = model.predict(padded_input_sequence)
    corrected_output_sequence = np.argmax(output_sequence, axis=-1)

    # Convert the corrected output sequence back to text
    corrected_output_text = tokenizer.sequences_to_texts(corrected_output_sequence)

    return corrected_output_text

In [ ]:
arabic_sentences = [
    "الحياه ف المدرسه ممتع",
    "ذهبت الئ المدرسه مبكن",
    "الولد يلعب في الشارعون",
    "اسمي يحي وانا اسكن في القاهر",
    "الطقسجميلو الشمستسطع",
    "رحه ممتع لمدينه جميله",
]
sentence_length=len (arabic_sentences)
prediction=[]
for i in range(sentence_length):
    corrected_text = predict(arabic_sentences[i])
    print("Corrected text:", corrected_text)
    prediction.append(corrected_text)

In [ ]:
sentences = [
    "اكلت طعما لذيذن في المطعم",
    "هزا الكتاب ممتع و مفيد",
    "الطلاب ذاهبون الي الجامعا",
    "احب مشاهدت الافلم في السينما",
    "العصافير تغرد علئ الشجره كل صباحن",
    "انه يكتب القصه جيدا و باسلوب جمل",
    "قرات مقال عن التكنلوجيا الحديثه",
]

prediction = []
for i in range(len(sentences)):
    corrected_text = predict(sentences[i])
    print("Corrected text:", corrected_text)
    prediction.append(corrected_text)


In [74]:
sentences = [
    "لأنة كتب الحملة  "
]

prediction = []
for i in range(len(sentences)):
    corrected_text = predict(sentences[i])
    print("Corrected text:", corrected_text)
    prediction.append(corrected_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Corrected text: ['كتب']


In [ ]:
import sys
from PySide6.QtWidgets import (
    QApplication,
    QMainWindow,
    QWidget,
    QVBoxLayout,
    QTextEdit,
    QLabel,
    QStatusBar,
)
from PySide6.QtCore import Qt, QTimer
from PySide6.QtGui import QFont, QTextCursor


class ArabicAutoCorrectApp(QMainWindow):
    def __init__(self):
        super().__init__()
        # GUI
        self.setWindowTitle("Arabic Auto-Correction")
        self.setGeometry(100, 100, 800, 600)

        self.timer = QTimer()
        self.timer.setSingleShot(True)
        self.timer.timeout.connect(self.process_text)

        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        layout = QVBoxLayout(central_widget)
        layout.setContentsMargins(20, 20, 20, 20)
        layout.setSpacing(20)

        title_label = QLabel("Arabic Text Auto-Correction")
        title_label.setFont(QFont("Helvetica", 24, QFont.Weight.Bold))
        title_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(title_label)

        self.text_edit = QTextEdit()
        self.text_edit.setFont(QFont("Arial", 14))
        self.text_edit.setAlignment(Qt.AlignRight)
        self.text_edit.setLayoutDirection(Qt.RightToLeft)
        self.text_edit.textChanged.connect(self.on_text_change)
        layout.addWidget(self.text_edit)

        self.status_bar = QStatusBar()
        self.setStatusBar(self.status_bar)
        self.status_bar.showMessage("Ready")
        self.text_edit.setFocus()
        #

        self.last_processed_text = ""

    def on_text_change(self):
        self.status_bar.showMessage("Processing...")
        self.timer.start(1000)

    def process_text(self):
        current_text = self.text_edit.toPlainText()

        if current_text != self.last_processed_text:
            corrected_text = predict(current_text)

            if corrected_text != current_text and corrected_text != "":
                self.text_edit.setPlainText(corrected_text)

                cursor = self.text_edit.textCursor()
                cursor.movePosition(QTextCursor.MoveOperation.End)
                self.text_edit.setTextCursor(cursor)

                self.text_edit.setFocus()

            if (corrected_text != ""):
                self.last_processed_text = corrected_text

        self.status_bar.showMessage("Ready")


def main():
    if QApplication.instance() is None:
        app = QApplication(sys.argv)
    else:
        app = QApplication.instance()

    app.setStyle("Fusion")
    window = ArabicAutoCorrectApp()
    window.show()
    sys.exit(app.exec())

In [ ]:
main()